In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os, sys
sys.path.append('/home/hrai/codes/hpe_library')
from lib_import import *
from my_utils import BatchCamera, Camera
from my_utils import arg_as_list, get_configs
from my_utils import generate_random_segment, generate_random_trajectory, get_aligned_init_torso, get_ap_pose_2d, get_backbone_line_from_torso, get_bounded_segments_idx, get_cam_param, get_input, get_label, get_model_input, get_output, get_pairs, get_part_traj, get_pose_seq_and_cam_param, get_two_point_parts, load_h36m, load_segment_file_from_parameters, make_input, make_one_dimension_list, make_output, split_continuous_indices, MyCustomDataset, parse_args_by_model_name, get_input_gt_for_onevec, get_limb_angle, get_h36m_camera_info, h36m_data_processing
from my_utils import DH_matrix, azim_elev_to_vec, batch_azim_elev_to_vec, batch_inverse_tf, batch_projection, batch_rot_x_matrix, batch_rot_y_matrix, batch_rot_z_matrix, calculate_azimuth_elevation, calculate_rotation_quaternion, draw_arm, draw_subline, draw_torso_direction, frame_vec_to_matrix, generate_batch_tf_from_batch_origin_R, generate_camera_frame, generate_dh_frame, generate_tf_from_origin_R, generate_two_link, generate_vis_frame_from_R, generate_world_frame, get_batch_frame_vec_from_keypoints, get_batch_lower_torso_frame_from_keypoints, get_batch_lower_torso_frame_from_pose, get_batch_reference_frame, get_batch_upper_torso_frame_from_keypoints, get_batch_upper_torso_frame_from_pose, get_frame_from_keypoints, get_lower_torso_frame_from_pose, get_reference_frame, get_torso_direction, get_torso_rotation_matrix, get_torso_shape, get_upper_torso_frame_from_pose, inverse_tf, normalize_vector, project_batch_tensor, projection, rotate_torso_by_R, rotate_torso_by_R_for_batch_tensor, rotation_matrix_to_vector_align, rotation_matrix_torso2torso, Appendage, BatchAppendage, BatchDHModel, DHModel, calculate_batch_azimuth_elevation
from my_utils import args_dict_to_namespace, construct_torso_from_output, denormalize_motionbert_result, get_dataset_info_from_segment_folder, get_inference_from_dhdst, get_inference_from_dhdst_torso, get_inference_from_motionbert, get_output_type, get_result, infer_one_segment, load_best_model_for_inference, normalize_input, test_model_by_segment_file
from my_utils import get_logger, log_configs
from my_utils import init_weights, split_array_by_idxs, BaselineModel, Linear, TorsoModel
from my_utils import Camera2ImageCoordinate, MPJPE, MPJPE_for_multiple_pose, MPJPE_for_single_pose, World2CameraCoordinate, World2ImageCoordinate, _sqrt_positive_part, _weak_project, aihub2h36m, array2dict, avgErrorForOneAction, avgErrorForOneActor, avgErrorForOneCamera, camera_to_image_frame, change_bbox_convention, check_max_min, coco2h36m, convert3DResult, draw_skeleton, draw_skeleton_2d, draw_skeleton_both, euclidean_distance, fit3d2h36m, fit3d_load_gt_and_param, get3DResult, getAIHubCameraParameter, getGT, getNumFromImgName, get_batch_h36m_keypoints, get_bbox_area, get_bbox_area_from_pose2d, get_bbox_from_pose2d, get_h36m_keypoint_index, get_h36m_keypoints, get_rootrel_pose, get_video_frame, get_video_info, get_xy_centered_pose, halpe2h36m, infer_box, kookmin2h36m, kookmin2h36m_with_nose, loadAIHubCameraParameter, matrix_to_quaternion, normalize, normalize_array, optimize_scaling_factor, plot_cv2_image, readJSON, readpkl, savepkl, skew_symmetric_matrix, skew_symmetric_matrix_tensor, standardize_quaternion, mpi_inf_3dhp2h36m, get_pose_height, get_batch_bbox_from_pose2d, len_ids, part_ids, h36m_part_keypoints, get_h36m_limb_lens
from my_utils import check_duplicate_training, get_input_output_candidate, get_num_trial, load_args, load_best_model, load_dataset, load_model, prepare_training, run, run_epoch, save_args, split_array_by_idxs
from my_utils import axes_2d, axes_3d, axes_to_compare_pred_gt, draw_3d_pose, draw_bbox, draw_multiple_2d_pose, draw_multiple_3d_pose, draw_one_segment, draw_rotation_matrix, draw_segment, draw_segments, draw_trajectory, get_2d_pose_image, legend_without_duplicate_labels, plot_to_compare_pred_gt, save_h36m_pose_video, show2Dtrajectory, show3Dtrajectory, show_2d_3d, show_whole_segment_trajectories, draw_2d_pose, clear_axes
from my_utils import check_available_frame, check_continuity, draw_base_marker_3d, generate_kookmin_pkl_for_each_video, get_cam_param_kookmin, get_video_frame_kookmin, get_video_num_frame_kookmin, load_pose3d_kookmin, load_csv_kookmin, get_lbot
os.chdir('/home/hrai/codes/MotionBERT')

from lib.utils.tools import * # get_config
from lib.utils.learning import * # load_backbone
from lib.utils.args import parse_args, check_args
from lib.utils.utils_data import flip_data
from lib.model.loss import *
from lib.model.evaluation import *
from lib.data.datareader_kookmin import DataReaderKOOKMIN
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_random_limb import DataReaderRandomLimb
from lib.data.dataset_motion_3d import MotionDataset3D
from lib.model.DHDSTformer import DHDSTformer_total, DHDSTformer_total2, DHDSTformer_total3, DHDSTformer_limb, DHDSTformer_torso, DHDSTformer_torso2

%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import threading
from ipywidgets import GridspecLayout
from ipywidgets import TwoByTwoLayout


box_layout = widgets.Layout(
        border='solid 1px red',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px')
 
#plt.clf()

class analysis_tool():
    def __init__(self):
        # variables -------------------------------------------------------------------
        self.stop_inference_flag = False
        self.inference_complete = False
        # model, data preparation -----------------------------------------------------
        self.load_panel()
        # inference -------------------------------------------------------------------
        self.inference_panel()
        # vis_control -----------------------------------------------------------------
        self.vis_control_panel()
        # figure ----------------------------------------------------------------------
        self.figure_panel()
        # analysis --------------------------------------------------------------------
        self.analysis_panel()
        # layout ----------------------------------------------------------------------
        ui_prepare = widgets.VBox([self.model_list, self.dataset_list, self.out_model_load, self.out_dataset_load], layout=box_layout, width='auto')
        ui_inference = widgets.VBox([self.button_inference, self.inference_progress, self.result_out, self.inputs_all_shape, self.gts_all_shape, self.results_all_shape], 
                                    layout=box_layout, width='auto')
        ui_vis_control = widgets.VBox([self.vis_interact, self.button_reset, self.play_vis], layout=box_layout, width='auto')
        ui_figure = widgets.HBox([self.plot_out, ui_vis_control])
        ui_analysis_control = widgets.VBox([self.error_max_frame, self.button_go_to_max_frame], layout=box_layout, width='auto')
        ui_analysis = widgets.Box([self.analysis_out, ui_analysis_control])
        ui_up = widgets.HBox([ui_prepare, ui_inference])
        self.ui = widgets.VBox([ui_up, ui_figure, ui_analysis])
        
    # panel functions -------------------------------------------------------------------  
    def load_panel(self):
        self.model_list = widgets.Dropdown(
            options=['MB_release', 'MB_ft_h36m', 'MB_train_h36m_with_canonical2', 'MB_ft_h36m_dh_angle1', 'MB_ft_h36m_dh_angle1_with_canonical', 'MB_ft_h36m_no_factor'],
            value='MB_ft_h36m',
            description='Model:',
            disabled=False,
        )

        self.dataset_list = widgets.Dropdown(
            options=['H36M', 'FIT3D', 'KOOKMIN', 'H36M_no_factor'],
            value='H36M',
            description='Dataset:',
            disabled=False,
        )
        self.out_model_load = widgets.interactive_output(self.load_model, {'target_model': self.model_list})
        self.out_dataset_load = widgets.interactive_output(self.load_data, {'target_dataset': self.dataset_list})
        
    def inference_panel(self):
        self.button_inference = widgets.Button(
            description='Inference',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='play'
        )
        self.button_inference.on_click(self.inference)

        self.result_out = widgets.Output()      
        self.inference_progress = widgets.FloatProgress(
            value=0,
            min=0,
            max=100,
            step=0.1,
            description='Testing:',
            bar_style='info',
            orientation='horizontal'
        )
        
        self.inputs_all_shape = widgets.Text(
            value='',
            placeholder='no data',
            description='inputs_all:',
            disabled=True
        )
        self.gts_all_shape = widgets.Text(
            value='',
            placeholder='no data',
            description='gts_all:',
            disabled=True
        )
        self.results_all_shape = widgets.Text(
            value='',
            placeholder='no data',
            description='results_all:',
            disabled=True
        )
        
    def vis_control_panel(self):
        self.batch_num = widgets.BoundedIntText(
            value=0,
            min=0,
            max=0,
            step=1,
            description='Batch num:',
            disabled=False
        )
        self.frame_num = widgets.BoundedIntText(
            value=1,
            min=1,
            max=1,
            step=1,
            description='Frame:',
            disabled=False
        )
        self.button_visualize = widgets.ToggleButton(
            value=False,
            description='Visualize',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            icon='check'
        )
        self.button_root_rel = widgets.ToggleButton(
            value=False,
            description='Root Rel',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            icon='check'
        )
        self.button_analysis = widgets.ToggleButton(
            value=False,
            description='Analysis',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            icon='check'
        )
        self.azim = widgets.IntSlider(
            value=-90,
            min=-180,
            max=180,
            step=1,
            description='azim:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
        self.elev = widgets.IntSlider(
            value=0,
            min=-180,
            max=180,
            step=1,
            description='elev:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
        self.zoom = widgets.FloatSlider(
            value=5.0,
            min=0.1,
            max=10,
            step=0.1,
            description='zoom:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='f'
        )
        self.play_vis = widgets.Play(
            #interval=10,
            value=1,
            min=1,
            max=1,
            step=1,
            description="Press play",
            disabled=False
        )
        widgets.jslink((self.play_vis, 'value'), (self.frame_num, 'value'))
        self.button_reset = widgets.Button(
            description='Reset',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='refresh'
        )
        self.button_reset.on_click(self.reset)
        self.vis_interact = interactive(self.visualize, batch_num=self.batch_num ,frame_num=self.frame_num, azim=self.azim, elev=self.elev, zoom=self.zoom, 
                                        visualize=self.button_visualize, root_rel=self.button_root_rel, analysis=self.button_analysis)
        #self.analysis_interact = interactive(self.analysis_error, batch_num=self.batch_num, frame_num=self.frame_num, root_rel=self.button_root_rel, analysis=self.button_analysis)
        
    def figure_panel(self):
        self.plot_out = widgets.Output(layout=box_layout, width='auto')
        with self.plot_out:
            self.fig = plt.figure(0, figsize=(15, 5))
            self.fig.clear()
            self.ax_input = axes_2d(self.fig, loc=131, normalize=True)
            self.ax_3d1 = axes_3d(self.fig, loc=132, xlim=(-1,1), ylim=(-1,1), zlim=(-1,1), view=(0,0), show_axis=False)
            self.ax_3d2 = axes_3d(self.fig, loc=133, xlim=(-1,1), ylim=(-1,1), zlim=(-1,1), view=(0,90), show_axis=False)
            f = plt.show()

        
    def analysis_panel(self):
        self.error_max_frame = widgets.Text(
            value='',
            placeholder='no data',
            description='max error frame:',
            disabled=True
        )
        
        self.button_go_to_max_frame = widgets.Button(
            description='Go to max frame',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        self.button_go_to_max_frame.on_click(self.go_to_max_frame)
        
        self.analysis_out = widgets.Output(layout=box_layout, width='auto')
        with self.analysis_out:
            self.fig2 = plt.figure(1, figsize=(15, 5), layout='constrained')
            self.fig2.clear()
            self.ax_error_per_frame = self.fig2.add_subplot(211)
            self.ax_error_per_joint = self.fig2.add_subplot(212)
            
            self.ax_error_per_frame.set_title('Joint error per frame')
            self.ax_error_per_frame.set_xlabel('Frame')
            self.ax_error_per_joint.set_title('Joint error per joint')
            self.ax_error_per_joint.set_xlabel('Joint')
            
            f = plt.show()
            
    # button functions -------------------------------------------------------------------
        
    def reset(self, b):
        self.frame_num.value = 1
        self.azim.value = -90
        self.elev.value = 0
        
    def go_to_max_frame(self, b):
        self.frame_num.value = self.error_max_frame.value
        
    def load_model(self, target_model):
        opts = parse_args_by_model_name(target=target_model)
        args = get_config(opts.config)
        args = check_args(args, opts)
        print(f'model: {args.model} \n loc: {opts.evaluate}')
        
        if 'DHDST_2' == args.model: model_pos = DHDSTformer_total2(chk_filename='', args=args)
        elif 'DHDSTformer_torso' == args.model: model_pos = DHDSTformer_torso(chk_filename='', args=args)
        elif 'DHDSTformer_torso2' == args.model: model_pos = DHDSTformer_torso2(chk_filename='', args=args)
        elif 'DHDSTformer_limb' == args.model: model_pos = DHDSTformer_limb(chk_filename='', args=args)
        elif 'MB' in opts.evaluate: model_pos = load_backbone(args)
        
        model_pos = nn.DataParallel(model_pos)
        model_pos = model_pos.cuda()
        checkpoint = torch.load(opts.evaluate, map_location=lambda storage, loc: storage)
        model_pos.load_state_dict(checkpoint['model_pos'], strict=True)
        model_pos = model_pos.eval()
        print('Model Loaded!')
        self.model_pos = model_pos
        self.args = args

        # reinference for the changed model
        try:
            self.button_inference.click()
        except Exception as e:
            pass
        
    def load_data(self, target_dataset):
        args = self.args
        if target_dataset == 'H36M':
            args.dt_file = 'h36m_sh_conf_cam_source_final.pkl'
            args.subset_list = ['H36M-SH']
            self.datareader = DataReaderH36M(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file)
            testloader_params = {
                'batch_size': args.batch_size,
                'shuffle': False,
                'num_workers': 12,
                'pin_memory': True,
                'prefetch_factor': 4,
                'persistent_workers': True
            }
            self.test_dataset = MotionDataset3D(args, args.subset_list, 'test')
            self.test_loader = DataLoader(self.test_dataset, **testloader_params)
            print(f'{target_dataset} Loaded!')
        elif target_dataset == 'H36M_no_factor':
            args.dt_file = 'h36m_sh_with_no_factor.pkl'
            args.subset_list = ['H36M-SH_WITH_NO_FACTOR']
            self.datareader = DataReaderH36M(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file)
            testloader_params = {
                'batch_size': args.batch_size,
                'shuffle': False,
                'num_workers': 12,
                'pin_memory': True,
                'prefetch_factor': 4,
                'persistent_workers': True
            }
            self.test_dataset = MotionDataset3D(args, args.subset_list, 'test')
            self.test_loader = DataLoader(self.test_dataset, **testloader_params)
            print(f'{target_dataset} Loaded!')
        elif target_dataset == 'FIT3D':
            print('FIT3D is not implemented yet')
        elif target_dataset == 'KOOKMIN':
            print('KOOKMIN is not implemented yet')
            
        # reinference for the changed dataset
        try:
            self.button_inference.click()
        except Exception as e:
            pass
        
    # def start_inference_thread(self, b):
    #     self.inference_thread = threading.Thread(target=self.inference)
    #     self.inference_thread.start()
    
    # def stop_inference(self, b):
    #     self.stop_inference_flag = True
    #     # self.result_out.clear_output()
    #     # with self.result_out:
    #     #     print(f'Stop button clicked: {self.stop_inference_flag}')
    #     with self.result_out:
    #         print(f'results_all: {self.results_all.shape}')
    #         print(f'gts_all: {self.gts_all.shape}')
    #         print(f'inputs_all: {self.inputs_all.shape}')
    #         self.inference_complete = False
            
    def inference(self, b):
        self.result_out.clear_output()
        args = self.args
        model_pos = self.model_pos
        test_loader = self.test_loader

        model_pos.eval() 
        save_path = f'/home/hrai/codes/MotionBERT/saved_results/result_dict_{args.model}_{args.subset_list[0]}.pkl'
        if os.path.exists(save_path):
            with self.result_out:
                print('Loading saved results...')
            result_dicts = readpkl(save_path)
            self.results_all = result_dicts['results_all']
            self.gts_all = result_dicts['gts_all']
            self.inputs_all = result_dicts['inputs_all']
        else:
            results_all = []
            gts_all = []
            inputs_all = []
            num_total_batch = len(test_loader)
            batch_step = 0
            with torch.no_grad():
                for batch_input, batch_gt in test_loader: # batch_input: normalized joint_2d, batch_gt: normalized joint3d_image
                    batch_size = len(batch_input) 
                    # preprocessing
                    batch_input, batch_gt, batch_gt_torso, batch_gt_limb_pos = preprocess_eval(args, batch_input, batch_gt)
                    # inference
                    predicted_3d_pos = batch_inference_eval(args, model_pos, batch_input, batch_gt_torso, batch_gt_limb_pos)
                    # postprocessing
                    if args.rootrel:
                        predicted_3d_pos[:,:,0,:] = 0     # [N,T,17,3]
                    else:
                        batch_gt[:,0,0,2] = 0
                    if args.gt_2d: # input 2d를 추론값으로 사용함으로써 depth만 추정하도록 함
                        predicted_3d_pos[...,:2] = batch_input[...,:2]
                    # store the results
                    results_all.append(predicted_3d_pos.cpu().numpy())
                    gts_all.append(batch_gt.cpu().numpy())
                    inputs_all.append(batch_input.cpu().numpy())
                        
                    batch_step += 1
                    batch_percentage = (batch_step/num_total_batch)*100
                    self.inference_progress.value = batch_percentage
                    self.inference_progress.description = f'{batch_percentage:.2f}%'

                    # if self.stop_inference_flag:
                    #     self.stop_inference_flag = False
                    #     break
                    #break
            self.results_all = np.concatenate(results_all)
            self.gts_all = np.concatenate(gts_all)
            self.inputs_all = np.concatenate(inputs_all)
            
            results_dict = {'results_all': self.results_all, 'gts_all': self.gts_all, 'inputs_all': self.inputs_all}
            savepkl(results_dict, save_path)
            with self.result_out:
                print('result saved!')
            
        with self.result_out:
            print(f'Inference Done for {args.model}!')

        # get clip info
        num_test_frames, action_clips, factor_clips, source_clips, frame_clips, gt_clips = get_clip_info(self.datareader, self.results_all)
        # print(action_clips.shape) # (B, T)
        # print(factor_clips.shape) # (B, T)
        self.action_clips = action_clips
        self.factor_clips = factor_clips
            
        # update
        self.results_all_shape.value = str(self.results_all.shape)
        self.gts_all_shape.value = str(self.gts_all.shape)
        self.inputs_all_shape.value = str(self.inputs_all.shape)
        self.batch_num.max = self.results_all.shape[0]-1
        self.frame_num.max = self.results_all.shape[1]
        self.play_vis.max = self.results_all.shape[1]

        self.vis_interact.update()
        #self.visualize(1)
    
    # def analysis_error(self, batch_num, frame_num, root_rel, analysis):
    #     try:
    #         frame_num -= 1
    #         batch_input = self.inputs_all[batch_num].copy()
    #         batch_gt = self.gts_all[batch_num].copy()
    #         batch_pred = self.results_all[batch_num].copy()
    #         if root_rel:
    #             batch_gt = batch_gt - batch_gt[:, 0].reshape(-1, 1, 3)
    #             batch_pred = batch_pred - batch_pred[:, 0].reshape(-1, 1, 3)
            
            
    #     except Exception as e:
    #         pass

    def visualize(self, batch_num, frame_num, azim, elev, zoom, visualize, root_rel, analysis):
        try: 
            frame_num -= 1
            batch_input = self.inputs_all[batch_num].copy()
            batch_gt = self.gts_all[batch_num].copy()
            batch_pred = self.results_all[batch_num].copy()
            if root_rel:
                batch_gt = batch_gt - batch_gt[:, 0].reshape(-1, 1, 3)
                batch_pred = batch_pred - batch_pred[:, 0].reshape(-1, 1, 3)
            if visualize:
                input_2d = batch_input[frame_num].copy()
                gt = batch_gt[frame_num].copy()
                pred = batch_pred[frame_num].copy()
                print(f'frame_num: {frame_num}')
                clear_axes([self.ax_input, self.ax_3d1, self.ax_3d2])
                
                # pelvis = gt[0]
                # min_x = pelvis[0] - 0.5
                # max_x = pelvis[0] + 0.5
                # min_y = pelvis[1] - 0.5
                # max_y = pelvis[1] + 0.5
                # min_z = pelvis[2] - 0.5
                # max_z = pelvis[2] + 0.5
                # ax_gt.set_xlim(min_x, max_x)
                # ax_gt.set_ylim(min_y, max_y)
                # ax_gt.set_zlim(min_z, max_z)
                self.ax_3d1.azim = azim
                self.ax_3d1.elev = elev
                self.ax_3d1.set_xlim(-1/zoom, 1/zoom)
                self.ax_3d1.set_ylim(-1/zoom, 1/zoom)
                self.ax_3d1.set_zlim(-1/zoom, 1/zoom)
                self.ax_3d1.set_aspect('equal')
                self.ax_3d1.set_title(f'batch {batch_num} frame {frame_num} GT - action: {self.action_clips[batch_num, frame_num]}')
                
                self.ax_3d2.azim = azim + 90
                self.ax_3d2.elev = elev
                self.ax_3d2.set_xlim(-1/zoom, 1/zoom)
                self.ax_3d2.set_ylim(-1/zoom, 1/zoom)
                self.ax_3d2.set_zlim(-1/zoom, 1/zoom)
                self.ax_3d2.set_aspect('equal')
                
                gt = rotate_torso_by_R(gt, Rotation.from_rotvec([-np.pi/2, 0, 0]).as_matrix())
                pred = rotate_torso_by_R(pred, Rotation.from_rotvec([-np.pi/2, 0, 0]).as_matrix())
                
                draw_2d_pose(self.ax_input, input_2d, normalize=True)
                draw_3d_pose(self.ax_3d1, gt, alpha=0.5, color='0')
                draw_3d_pose(self.ax_3d1, pred)
                draw_3d_pose(self.ax_3d2, gt, alpha=0.5, color='0')
                draw_3d_pose(self.ax_3d2, pred)
            if analysis:
                # calculate mpjpe and pa-mpjpe
                print(f'batch_gt: {batch_gt.shape}', f'batch_pred: {batch_pred.shape}')
                error = np.linalg.norm(batch_gt - batch_pred, axis=-1) # [243, 17]
                frame_level_error = error.mean(axis=-1) # [243]
                frame_level_var = error.var(axis=-1) # [243]
                joint_level_error = error.mean(axis=0) # [17]
                joint_level_var = error.std(axis=0) # [17]
                joint_error_at_frame = error[frame_num] # [17]
                
                print(f'frame_level_error: {frame_level_error.shape}')
                print(f'frame_level_var: {frame_level_var.shape}')
                print(f'joint_level_error: {joint_level_error.shape}')
                print(f'joint_error_at_frame: {joint_error_at_frame.shape}')
                with self.analysis_out:
                    clear_axes([self.ax_error_per_frame, self.ax_error_per_joint])
                    self.ax_error_per_frame.plot(frame_level_error, 'r')
                    self.ax_error_per_frame.set_ylim(0, frame_level_error.max()*1.2)
                    self.ax_error_per_frame.vlines(frame_num, 0, frame_level_error.max()*1.2, colors='b', linestyles='dashed')

                    self.ax_error_per_joint.bar(np.arange(len(joint_level_error)), joint_level_error, yerr=joint_level_var, color='r')
                    self.ax_error_per_joint.bar(np.arange(len(joint_error_at_frame)), joint_error_at_frame, width=0.5, alpha=0.5, color='b')
                    self.ax_error_per_joint.set_xticks(np.arange(len(joint_error_at_frame)))
                    self.ax_error_per_joint.set_ylim(0, joint_level_error.max()*2)
                self.error_max_frame.value = str(frame_level_error.argmax())

                    
        except Exception as e:
            print(e)
        
at = analysis_tool()
at.button_inference.click()
at.button_visualize.value = True
at.button_root_rel.value = True
at.button_analysis.value = True
a = display(at.ui)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
